In [1]:
!nvidia-smi

Wed Aug 12 01:20:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.67       Driver Version: 451.67       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce MX150      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8    N/A /  N/A |     64MiB /  2048MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [2]:
!python main.py videoed extract-video --input-file workspace/data_dst.* --output-dir workspace/data_dst/ --fps 0 --output-ext jpg

usage: main.py [-h]
               {extract,sort,util,train,merge,videoed,facesettool,dev_test,xseg}
               ...

positional arguments:
  {extract,sort,util,train,merge,videoed,facesettool,dev_test,xseg}
    extract             Extract the faces from a pictures.
    sort                Sort faces in a directory.
    util                Utilities.
    train               Trainer
    merge               Merger
    videoed             Video processing.
    facesettool         Faceset tools.
    dev_test
    xseg                XSeg tools.

optional arguments:
  -h, --help            show this help message and exit


In [ ]:
!python main.py videoed extract-video --input-file workspace/data_src.* --output-dir workspace/data_src/ --fps 0 --output-ext jpg

# Denoise Frames

In [ ]:
!python main.py videoed denoise-image-sequence --input-dir workspace/data_dst --factor 7

In [ ]:
!python main.py videoed denoise-image-sequence --input-dir workspace/data_src --factor 7

# Detect Faces and Align

In [1]:
#@markdown Select the Type of **Detector** (s3fd recommended) and **FaceType**
Detector = "s3fd" #@param ["s3fd", "manual"]
FaceType = "whole_face" #@param ["head", "half_face", "full_face", "whole_face", "mark_only"]

In [2]:
!python main.py extract --input-dir workspace/data_dst --output-dir workspace/data_dst/aligned --face-type whole_face --detector s3fd --force-gpu-idxs 0 --max-faces-from-image 0 --image-size 512 --jpeg-quality 90 --no-output-debug

^C


In [ ]:
!python main.py extract --input-dir workspace/data_src --output-dir workspace/data_src/aligned --face-type whole_face --detector s3fd --force-gpu-idxs 0 --max-faces-from-image 0 --image-size 512 --jpeg-quality 90 --no-output-debug

# Sort Aligned Faces

In [ ]:
#@markdown Select **sort type** (absdiff recommended)
sort_type = "absdiff" #@param ["absdiff", "brightness", "hist", "hist-dissim", "face-yaw", "face-pitch", "vggface", "blur", "final", "final-no-blur"]

In [ ]:
!python main.py sort --input-dir workspace/data_dst/aligned --by absdiff

In [ ]:
!python main.py sort --input-dir workspace/data_src/aligned --by absdiff

#  Faceset Enhance 

In [ ]:
!python main.py facesettool enhance --input-dir /workspace/data_dst/aligned

In [ ]:
!python main.py facesettool enhance --input-dir /workspace/data_src/aligned

# Train Model

In [ ]:
Model = "SAEHD" #@param ["SAEHD", "SAE", "Quick96", "H128", "LIAEF128", "DF"]

In [ ]:
!python main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model SAEHD

# Convert Frames

In [ ]:
!python main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model SAEHD

# Get Final Result Video

In [ ]:
!python main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4